#### YAPILMIŞTI DÜZENLENECEK (VAD ve FILTERDAN SONRA)

In [1]:
# Gerekli kütüphaneler
import librosa
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import warnings        
warnings.filterwarnings("ignore")

In [2]:
# Veri işleme fonksiyonu
def read_and_sample(csv_path, fraction):
    data = pd.read_csv(csv_path)
    sample_data = data.sample(frac=fraction, random_state=42)
    return sample_data

In [3]:
# Verileri oku
sample_fraction = 1

train_data = read_and_sample("Datasets/train.csv", sample_fraction)
val_data = read_and_sample("Datasets/val.csv", sample_fraction)
test_data = read_and_sample("Datasets/test.csv", sample_fraction)

In [4]:
# Verileri Kontrol et
print("Train Data Sample:")
print(train_data.head())
print("Validation Data Sample:")
print(val_data.head())
print("Test Data Sample:")
print(test_data.head())

Train Data Sample:
                                                features  labels
11456  Datasets\zh-CN\clips\common_voice_zh-CN_189554...       1
1288   Datasets\zh-CN\clips\common_voice_zh-CN_188230...       1
5535   Datasets\zh-CN\clips\common_voice_zh-CN_189779...       1
12657  Datasets\zh-CN\clips\common_voice_zh-CN_190086...       1
8603      Datasets\tr\clips\common_voice_tr_17527179.mp3       0
Validation Data Sample:
                                               features  labels
1652  Datasets\zh-CN\clips\common_voice_zh-CN_189997...       1
5303  Datasets\zh-CN\clips\common_voice_zh-CN_189834...       1
2987  Datasets\zh-CN\clips\common_voice_zh-CN_198784...       1
1545     Datasets\tr\clips\common_voice_tr_17529087.mp3       0
3767  Datasets\zh-CN\clips\common_voice_zh-CN_186595...       1
Test Data Sample:
                                               features  labels
1057     Datasets\tr\clips\common_voice_tr_17483907.mp3       0
812      Datasets\tr\clips\common_voi

In [5]:
# Etiket dönüşüm fonksiyonu
def labels(label):
    return tf.constant([1, 0] if label == 0 else [0, 1], dtype=tf.int32)

In [6]:
# train verileri yükle
train_x_list, train_y_list = [], []

for index, row in train_data.iterrows():
    audio, sample_rate = librosa.load(row['features'])
    audio_feature = librosa.feature.melspectrogram(y=audio*2, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(audio_feature, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    train_x_list.append(tensor_x)
    train_y_list.append(tensor_y)

In [7]:
# val verileri yükle
val_x_list, val_y_list = [], []

for index, row in val_data.iterrows():
    audio, sample_rate = librosa.load(row['features'])
    audio_feature = librosa.feature.melspectrogram(y=audio*2, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(audio_feature, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    val_x_list.append(tensor_x)
    val_y_list.append(tensor_y)

In [8]:
# Parametreleri ayarla
batch_size = 64
epochs = 1000
input_size = 128
hidden_size = 256
output_size = 2
learning_rate=1e-4

In [10]:
# train_dataset oluştur
def generator_train():
    for features, labels in zip(train_x_list, train_y_list):
        yield (features, labels)

train_dataset = tf.data.Dataset.from_generator(
    generator_train,
    output_types=(tf.float32, tf.int32),
    output_shapes=((None, input_size), (output_size))
)

train_dataset = train_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size],[output_size]))

In [11]:
# val_dataset oluştur
def generator_val():
    for features, labels in zip(val_x_list, val_y_list):
        yield (features, labels)
        
val_dataset = tf.data.Dataset.from_generator(
    generator_val,
    output_types=(tf.float32, tf.int32),
    output_shapes=((None, input_size), (output_size))
)

val_dataset = val_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size], [output_size]))

In [12]:
# Modeli oluştur
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, input_size), batch_size=batch_size),
    tf.keras.layers.LSTM(hidden_size,kernel_regularizer=tf.keras.regularizers.L2()),
    tf.keras.layers.Dense(output_size, activation='linear',kernel_regularizer=tf.keras.regularizers.L2())
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (64, 128)              │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 2)                │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,842 (515.01 KB)

 Trainable params: 131,842 (515.01 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Modeli eğit
checkpoint_path = "Checkpoints/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = model.fit(train_dataset,  
          epochs=epochs,
          validation_data=val_dataset,
          callbacks=[cp_callback])

Epoch 1/10
      4/Unknown 6s 575ms/step - accuracy: 0.5496 - loss: 2.7821
Epoch 1: saving model to Checkpoints/cp.weights.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5543 - loss: 2.7812 - val_accuracy: 0.5909 - val_loss: 2.7605
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 559ms/step - accuracy: 0.6155 - loss: 2.7559
Epoch 2: saving model to Checkpoints/cp.weights.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 734ms/step - accuracy: 0.6139 - loss: 2.7551 - val_accuracy: 0.5909 - val_loss: 2.7352
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 615ms/step - accuracy: 0.6155 - loss: 2.7303
Epoch 3: saving model to Checkpoints/cp.weights.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 770ms/step - accuracy: 0.6139 - loss: 2.7295 - val_accuracy: 0.5909 - val_loss: 2.7095
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.6155 - loss: 2.7044
Epoch 4: saving model to Checkpoints/cp.weights.h5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 719ms/step - accuracy: 0.6139 - loss: 2.7036 - val_accuracy: 0.5909 - val_loss: 2.6834
Epoch 5/10
4

In [ ]:
# Loss grafiği çizdir
loss = history.history['loss']
val_loss = history.history['val_loss']

fig, ax = plt.subplots(1, 1)

ax.plot(loss, label='Loss')
ax.plot(val_loss, label='Validation Loss')
ax.set_title('Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend()

plt.show()

In [ ]:
# Validation Accuracy grafiği çizdir
val_acc = history.history['val_accuracy']

fig, ax = plt.subplots(1, 1)

ax.plot(val_acc, label='Validation Accuracy')
ax.set_title('Validation Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.legend()

plt.show()

In [ ]:
# test verileri yükle
test_x_list, test_y_list = [], []

for index, row in test_data.iterrows():
    audio, sample_rate = librosa.load(row['features'])
    audio_feature = librosa.feature.melspectrogram(y=audio*2, sr=sample_rate)
    tensor_x = tf.transpose(tf.convert_to_tensor(audio_feature, dtype=tf.float32))
    tensor_y = labels(row['labels'])
    test_x_list.append(tensor_x)
    test_y_list.append(tensor_y)
    print(tensor_x.shape, tensor_y.shape)

In [ ]:
# test_dataset oluştur
def generator_test():
    for features, labels in zip(test_x_list, test_y_list):
        yield (features, labels)

test_dataset = tf.data.Dataset.from_generator(
    generator_test,
    output_types=(tf.float32, tf.int16),
    output_shapes=((None, input_size), (output_size))
)

test_dataset = test_dataset.padded_batch(batch_size=batch_size, padded_shapes=([None, input_size], [output_size]))

In [ ]:
loss, acc = model.evaluate(test_dataset, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))